# Training of the Neural Network based Auditory Model

## An Auditory Model for  Azimuthal Localisation in Sound Field Synthesis

In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, SGD
from keras.backend.tensorflow_backend import set_session

import tensorflow as tf

from utils.custom_loss import mae_wrap_angle, mse_wrap_angle
from utils.load_data_raw import DataGenerator_raw, load_raw_ft_h5, load_raw_IDs_h5
from utils.dataset_split import create_split

In [ ]:
NUM_WORKER = 4

# configure tf session; disable usage of GPU
config = tf.ConfigProto(device_count={'CPU': NUM_WORKER,
                                      'GPU': 0},
                        intra_op_parallelism_threads= NUM_WORKER, # num of cores per socket
                        inter_op_parallelism_threads=2,
                        allow_soft_placement=True)
sess = tf.Session(config=config)
set_session(sess)

# configuration as suggested here:
# https://software.intel.com/en-us/articles/tips-to-improve-performance-for-popular-deep-learning-frameworks-on-multi-core-cpus

## Model Definition

In [ ]:
def simple_model(in_dim):
    """Define simple model"""
    # create model
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(in_dim,)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    # show info
    model.summary()
    # compile model
    model.compile(loss = mae_wrap_angle,
                  optimizer = 'adam',
                  metrics = [mse_wrap_angle])
    return model

## Training

#### Set train parameters and load the data set

In [ ]:
# define training parameters
batch_size = 1024
num_epochs = 200

# define filename of file containing dataset
filename = '../../generated/database/raw/raw_nf10_mid/database_raw_nf10_scaledMM.h5'

# list of substrings of parameters to select, e.g. 'NFCHOA', 'pos10', 'R006'
valid_subset = [] #['LWFS','R006','M006','M027','R027','M013','R013']
test_subset = ['NFCHOA_R006', 'NFCHOA_M027']

In [ ]:
# load data indices and reference tables
ID_ref, pos_table, cond_table, par = load_raw_IDs_h5(filename)
# create test/validation/train dataset split
partition = create_split(ID_ref,cond_table,test_subset,valid_split=0.2)
# load feature and target data
feature_data, target_data, _ = load_raw_ft_h5(filename)

#### Train on batches of data set with data generator

In [ ]:
# define generators
params = {'dim': feature_data.shape[1],
          'batch_size': batch_size,
          'feature_data': feature_data.values,
          'target_data' : target_data.values,
          'shuffle': True,
          'n_frames': par['nFrames']
         }
train_batch_generator = DataGenerator_raw(partition['train'], **params)
valid_batch_generator = DataGenerator_raw(partition['validation'], **params)

# create model
model = simple_model(feature_data.shape[1])

# define callbacks
csv_logger = keras.callbacks.CSVLogger('log.csv')
e_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', mode='min', patience=20)
cb_list = [csv_logger, e_stop]

# start training
history = model.fit_generator(generator = train_batch_generator,
                              epochs = num_epochs,
                              verbose = 1,
                              validation_data = valid_batch_generator,
                              callbacks = cb_list,
                              use_multiprocessing = True,
                              workers = NUM_WORKER,
                              max_queue_size = 100
                              )

## Evaluation

#### For further evaluation, dedicated notebooks are available.

In [ ]:
# plot train history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss (mae w angle wrapping)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['mae_wrap_angle'])
plt.plot(history.history['val_mae_wrap_angle'])
plt.title('Model loss metric mse w angle wrapping')
plt.ylabel('MSE-W')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# evaluate model
test_batch_generator = DataGenerator_raw(partition['test'], **params)
score = model.evaluate_generator(test_batch_generator, verbose=1)
print('Test loss:', score[0])
print('Test mse w wrap:', score[1])

## Save trained model to file

Model naming convention of this project:
 - m128-128_LOSS_OPTIMIZER_bsBATCHSIZE_t-TESTDATASET_PARAMETER_SCENARIO

In [ ]:
model_name = 'm128-128_maew_adam_bs1024_t-NR006-NM027'
# Save train history to json file
import json
with open('../data/models_trained/'+model_name+'_history.json', 'w') as f:
    json.dump(history.history, f)
    
# Save model
from keras.models import load_model
model.save('../data/models_trained/'+model_name+'.h5')
del model